In [1]:
%cd ..

/Users/chan/boostcamp-project/level2-nlp-generationfornlp-nlp-06-lv3


/Users/chan/boostcamp-project/level2-nlp-generationfornlp-nlp-06-lv3/.venv/lib/python3.11/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [ ]:
import os

import pandas as pd
from datasets import load_dataset
from dotenv import load_dotenv
from langchain.output_parsers import BooleanOutputParser, OutputFixingParser
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from tqdm import tqdm

from prompts import load_template

/Users/chan/boostcamp-project/level2-nlp-generationfornlp-nlp-06-lv3/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
load_dotenv()

PARAGRAPH = "paragraph"
QUESTION = "question"
CHOICES = "choices"
ANSWER = "answer"

VALID = "is_valid"

dataset = load_dataset("skt/kobest_v1", "boolq")

In [ ]:
train_df = pd.DataFrame(dataset["train"])
valid_df = pd.DataFrame(dataset["validation"])
test_df = pd.DataFrame(dataset["test"])

kobest_df = pd.concat([train_df, valid_df, test_df])
kobest_df.head()

,paragraph,question,label
0,로마 시대의 오리엔트의 범위는 제국 내에 동부 지방은 물론 제국 외부에 있는 다른 ...,"오리엔트는 인도와 중국, 일본을 이루는 광범위한 지역을 지칭하는 단어로 쓰인다.",1
1,비글을 키우려면 비글이 뛰어놀수 있는 넓은 놀이공간 등을 확보하고 있는 단독주택이 ...,비글은 넓고 뚤린 공간에서 키워야 한다.,1
2,타이완 요리의 특징은 토속 요리(일본 통치 전)에서 기름을 많이 사용하는 다른 지역...,타이완 요리는 다른 지역의 중국 요리처럼 기름을 많이 사용하는 것이다.,0
3,연하곤란은 음식물이 구강에서 식도로 넘어가는 과정에 문제가 생겨 음식을 원활히 혹은...,연하곤란이 생기면 식도가 막히나요?,0
4,"인문과학 또는 인문학(人文學, 영어: humanities)은 인간과 인간의 근원문제...",인문과학은 경험적인 접근을 주로 사용하는가?,0


In [ ]:
generation_prompt = PromptTemplate.from_template(
    load_template(file_name="question_choices_generation.txt", template_type="question_choices_generation")
)

llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    api_key=os.getenv("OPENAI_API_KEY"),
)

output_parser = JsonOutputParser()

fixing_parser = OutputFixingParser.from_llm(parser=output_parser, llm=llm)

problem_generator = generation_prompt | llm | fixing_parser

In [ ]:
def generate_problem(data: pd.Series):
    response = problem_generator.invoke({PARAGRAPH: data[PARAGRAPH]})
    return response[QUESTION], response[CHOICES], response[ANSWER]


tqdm.pandas()
sampled_df = kobest_df.sample(20, random_state=1004)
sampled_df["result"] = sampled_df.progress_apply(generate_problem, axis=1)

# 결과 분리
sampled_df[QUESTION] = sampled_df["result"].apply(lambda x: x[0])
sampled_df[CHOICES] = sampled_df["result"].apply(lambda x: x[1])
sampled_df[ANSWER] = sampled_df["result"].apply(lambda x: x[2])

# 임시 열 삭제
sampled_df.drop(columns=["result", "label"], inplace=True)

sampled_df.head()

100%|██████████| 20/20 [00:50<00:00,  2.55s/it]


,paragraph,question,choices,answer
1139,이스타나 왕궁은 싱가포르의 대통령궁이다. 싱가포르의 대표적인 중심가인 오차드 거리의...,"지문에 따르면, 이스타나 왕궁의 역사적 배경에 대한 설명으로 옳은 것은?","[이스타나 왕궁은 싱가포르의 독립 선언 이후에 지어진 건물이다., 왕궁은 한때 육두...",1
845,"찟라다 궁은 태국 방콕에 위치한 궁전으로 태국의 국왕, 왕비의 관저로 사용되고 있다...","지문에 따르면, 찟라다 궁이 태국의 국왕과 왕비의 관저로 사용되기 시작한 이유는 무...",[라마 9세 국왕이 방콕 왕궁에서 이곳으로 옮긴 이유는 그의 형의 의문의 죽음 때문...,1
728,DNA는 뉴클레오타이드의 중합체인 두 개의 긴 가닥이 서로 꼬여있는 이중나선 구조로...,"지문에 따르면, DNA의 주요 기능 중 하나는 무엇인가?","[DNA는 세포의 에너지를 생성하는 역할을 한다., DNA는 유전정보를 통해 유전자...",2
296,"브루나이 음식은 종종 매운 맛이 나며, 일반적으로 쌀이나 국수와 함께 먹는다. 브루...","지문에 따르면, 브루나이 음식의 특징으로 옳은 것은?","[브루나이 음식은 일반적으로 매운 맛이 나며, 쌀이나 국수와 함께 제공된다., 브루...",1
322,2009년 동국대학교 및 중앙대학교 연극영화과 수시 전형에 동시 합격한 뒤 최종 중...,"지문에 따르면, 유리가 중앙대학교에서 졸업식에서 수여받은 상의 이름은 무엇인가?","[최우수 학생상, 봉사공로상, 연예인 홍보대사상, 학업 우수상, 특별 공로상]",1


In [ ]:
sampled_df.to_csv("data/experiments/kobest_exp_v4.0.1.csv", index=False)

In [ ]:
validation_prompt = PromptTemplate.from_template(
    load_template(file_name="validation.txt", template_type="question_choices_generation")
)

llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    api_key=os.getenv("OPENAI_API_KEY"),
)

output_parser = BooleanOutputParser()

fixing_parser = OutputFixingParser.from_llm(parser=output_parser, llm=llm)

validator = validation_prompt | llm | fixing_parser

In [9]:
def validate_problem(data: pd.Series):
    response = validator.invoke(
        {PARAGRAPH: data[PARAGRAPH], QUESTION: data[QUESTION], CHOICES: data[CHOICES], ANSWER: data[ANSWER]}
    )
    return response


tqdm.pandas()
sampled_df = sampled_df.sample(20, random_state=1004)
sampled_df[VALID] = sampled_df.progress_apply(validate_problem, axis=1)

sampled_df.head()

100%|██████████| 20/20 [00:11<00:00,  1.76it/s]


,paragraph,question,choices,answer,is_valid
622,버섯 구름(mushroom cloud)은 응축된 수증기나 먼지로 구성된 버섯 형상의...,"지문에 따르면, 버섯 구름이 생성되는 주된 원인은 무엇인가?","[핵폭발과 같은 매우 큰 폭발의 결과로 형성된다., 주로 화산 폭발로 인해 자연적으...",1,True
2806,부르키나파소 요리는 서아프리카에 있는 부르키나파소의 요리이다. 전형적인 서아프리카 ...,"지문에 따르면, 부르키나파소 요리의 주요 재료로 사용되지 않는 것은 무엇인가?","[기장, 쌀, 옥수수, 닭고기, 오크라]",4,False
2310,서북공심돈의 성제(城制)는 우리나라 성곽에 처음 있는 것으로 화성성역의궤에 기록된 ...,"지문에 따르면, 서북공심돈의 성제의 특징으로 옳은 것은?","[서북공심돈은 우리나라 성곽에서 처음으로 발견된 구조물이다., 화성성역의궤에 기록된...",2,True
2163,입실렌티 지·야의 함성(Ipselenti: 知野의 喊聲)은 주로 석탑대동제 마지막 ...,"지문에 따르면, '입실렌티 지·야의 함성'의 의미와 기원은 무엇인가?","[입실렌티는 고려대학교의 교호의 일부로, 지성과 야성을 상징한다., 입실렌티는 보성...",1,False
3251,퓨마 또는 검은표범은 아메리카 대륙에 서식하는 펠리네아과에 속하는 큰 종이다. 캐나...,"지문에 따르면, 퓨마의 서식지와 특징에 대한 설명으로 옳은 것은?","[퓨마는 아메리카 대륙의 모든 지역에서 발견되며, 가장 큰 육상 포유류이다., 퓨마...",2,True


In [ ]:
sampled_df.to_csv("data/experiments/kobest_exp_v4.0.1.csv", index=False)